# Practica 2

## Ejercicio 1

sofmdosfm

## Ejercicio 2


In [2]:
import datetime,math,re
import random
import sys
sys.path.append('..')

from MRE import Job

root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje2/"
output_dir = output_path + "eje2/"


def fmap(key, value, context):
    context.write(value, 1)
    
def fcomb(key, values, context):
    c = 0
    for v in values: 
        c += v
    context.write(key, c)
    
def fred(key,values,context):
    n = 0
    for v in values:
        n += v
    context.write(key,n)

job = Job(input_dir,output_dir,fmap,fred)
job.setCombiner(fcomb)
success = job.waitForCompletion()
print(success)

True


## Ejercicio 3

Mi lógica 